In [1]:
# 1) install the BERTScore metric
!pip --quiet install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is inc

In [2]:
import pandas as pd
import os
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from bert_score import score
import time
from tqdm.auto import tqdm

2025-05-16 16:56:33.886422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747414594.100010      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747414594.163505      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 2) load your QA dataset
df = pd.read_csv("/kaggle/input/test-data-curation/qa_dataset.csv")

In [4]:
# 3) init BLIP-VQA baseline
device    = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model     = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [8]:
# 4) run inference loop
preds = []
refs  = []
image_root = "/kaggle/input/test-data-dataset/images"

start_time = time.time()

for _, row in tqdm(df.iterrows(), total=len(df), desc="BLIP-VQA Inference"):
    img_path = os.path.join(image_root, row["path"])
    image    = Image.open(img_path).convert("RGB")
    inputs   = processor(image, row["question"], return_tensors="pt").to(device)
    out      = model.generate(**inputs)
    pred     = processor.decode(out[0], skip_special_tokens=True).strip()
    
    preds.append(pred)
    refs.append(str(row["answer"]).strip())

end_time = time.time()

BLIP-VQA Inference:   0%|          | 0/5994 [00:00<?, ?it/s]

In [13]:
# 4) compute metrics
total_time    = end_time - start_time
avg_time_ms   = total_time / len(preds) * 1000

# Exact‐Match Accuracy
exact_match = sum(p.lower() == r.lower() for p, r in zip(preds, refs)) / len(preds)

# BERTScore F1 (all preds)
P_all, R_all, F1_all = score(preds, refs, lang="en", rescale_with_baseline=True)
bert_f1_all = F1_all.mean().item()

# One‐word analysis
is_one_word   = [((" " not in p) and p != "") for p in preds]
pct_one_word  = sum(is_one_word) / len(preds) * 100
preds_1w      = [p for p, ok in zip(preds, is_one_word) if ok]
refs_1w       = [r for r, ok in zip(refs,  is_one_word) if ok]

if preds_1w:
    _, _, F1_1w = score(preds_1w, refs_1w, lang="en", rescale_with_baseline=True)
    bert_f1_1w = F1_1w.mean().item()
else:
    bert_f1_1w = float("nan")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# 5) report
print(f"Total inference time: {total_time:.1f}s")
print(f"Average per sample:                {avg_time_ms:.1f}ms")
print()
print(f"Exact-Match Accuracy:              {exact_match:.2f}")
print(f"BERTScore F1 (all predictions):    {bert_f1_all:.2f}")
print()
print(f"% One-Word Predictions:            {pct_one_word:.2f}%")
print(f"BERTScore F1 (one-word only):      {bert_f1_1w:.2f}")

Total inference time: 538.9s
Average per sample:                89.9ms

Exact-Match Accuracy:              0.58
BERTScore F1 (all predictions):    0.81

% One-Word Predictions:            95.51%
BERTScore F1 (one-word only):      0.85
